In [ ]:
import numpy as np
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, circle
from pathlib import Path

%pylab inline

from skimage.io import imread, imshow, imsave

In [ ]:
def overlap(x1, y1, r1, x2, y2, r2):
    """
        Check if distance between two circle centres is less 
        than the sum of the radii (they are overlapping)
    
        :param x1: X-coordinate of circle 1
        :param y1: Y-coordinate of circle 1
        :param r1: radius of circle 1
        :param x2: X-coordinate of circle 2
        :param y2: Y-coordinate of circle 2
        :param r2: radius of circle 2
        :type x1: float
        :type y1: float
        :type r1: float
        :type x2: float
        :type y2: float
        :type r2: float
        :rtype: bool
        :return: Are two circles overlapping
    """
    dist = sqrt((x1 - x2)**2 + (y1 - y2)**2) * 1.05
    return dist < r1 + r2

In [ ]:
in_dir = Path('/PATH/TO/INPUT/FOLDER/')

out_dir = Path('/PATH/TO/LABEL/OUTPUT/FOLDER/')
out_dir.mkdir(exist_ok=True)

outline_dir = Path('/PATH/TO/OUTLINE/OUTPUT/FOLDER/')
outline_dir.mkdir(exist_ok=True)

In [ ]:
for f in sorted(in_dir.glob('*tif')):
    im = imread(f, as_gray=True)
    edges = canny(im)
    
    imshow(edges)
    gcf().set_size_inches(10, 10)
    show()
    
    hough_radii = np.arange(20, 120, 3)  # adjust parameters for batch of images (min, max radius; step size)
    hough_res = hough_circle(edges, hough_radii)
    
    accums, cx, cy, radii = hough_circle_peaks(
        hough_res, 
        hough_radii, 
        total_num_peaks=40,  # Expected number of cells in image
        min_ydistance=20,  # Expected minimum cell radius (in pixels)
        min_xdistance=20)  # Expected minimum cell radius (in pixels)

    accepted = []
    mask = zeros(im.shape, dtype=np.int8)
    label = 1

    for center_y, center_x, radius in zip(cy, cx, radii):
        good = True

        for accy, accx, accr in accepted:
            if overlap(center_x, center_y, radius, accx, accy, accr):
                good = False

        if not good:
            continue

        accepted.append((center_y, center_x, radius))

        circy, circx = circle_perimeter(center_y, center_x, radius, shape=im.shape)
        im[circy, circx] = 1

        masky, maskx = circle(center_y, center_x, radius, shape=im.shape)
        mask[masky, maskx] = label
        label += 1

    imshow(im)
    gcf().set_size_inches(10, 10)
    show()
    imsave(outline_dir / (f.stem + '.png'), im)
    
    imsave(out_dir / f.name, mask)